In [48]:
import pandas as pd
import sqlite3
import os

def load_energy_data(db_path="database.db"):
    # Connect to the database
    conn = sqlite3.connect(db_path)

    try:
        # Load all tables
        devices_df = pd.read_sql_query("SELECT * FROM devices", conn)

        historical_df = pd.read_sql_query("SELECT * FROM historical_energy_readings", conn)
        historical_df["timestamp"] = pd.to_datetime(historical_df["timestamp"])

        realtime_df = pd.read_sql_query("SELECT * FROM real_time_energy_readings", conn)
        realtime_df["timestamp"] = pd.to_datetime(realtime_df["timestamp"])

        predictions_df = pd.read_sql_query("SELECT * FROM predictions", conn)
        predictions_df["timestamp"] = pd.to_datetime(predictions_df["timestamp"])

        scheduled_tasks_df = pd.read_sql_query("SELECT * FROM scheduled_tasks", conn)
        scheduled_tasks_df["scheduled_time"] = pd.to_datetime(scheduled_tasks_df["scheduled_time"], errors='coerce')

        anomalies_df = pd.read_sql_query("SELECT * FROM anomalies", conn)
        anomalies_df["timestamp"] = pd.to_datetime(anomalies_df["timestamp"])

        # Group by switch_id and count anomalies
        anomaly_counts = anomalies_df.groupby("switch_id").size().reset_index(name="anomaly_count")

    finally:
        conn.close()

    # Return all DataFrames in a dictionary
    return {
        "devices": devices_df,
        "historical": historical_df,
        "realtime": realtime_df,
        "predictions": predictions_df,
        "scheduled_tasks": scheduled_tasks_df,
        "anomalies": anomalies_df,
        "anomaly_counts": anomaly_counts
    }

In [49]:
data = load_energy_data()

data["devices"]
data["historical"]
data['realtime']
data['predictions']
data['scheduled_tasks']
# data['anomalies']
# data['anomaly_counts']

,task_id,switch_id,target_date,scheduled_time,status


In [50]:
data["historical"].head(60)

,switch_id,timestamp,power_consumption
0,ac_01,2025-04-16 11:58:08,0.00
1,ac_02,2025-04-16 11:58:08,0.00
2,ac_03,2025-04-16 11:58:08,346.88
3,ac_04,2025-04-16 11:58:08,247.79
4,mw_01,2025-04-16 11:58:08,0.00
5,rf_01,2025-04-16 11:58:08,67.91
6,dw_01,2025-04-16 11:58:08,0.00
7,sp_01,2025-04-16 11:58:08,0.00
8,sp_02,2025-04-16 11:58:08,0.00
9,sp_03,2025-04-16 11:58:08,0.00


In [51]:
data["historical"]

,switch_id,timestamp,power_consumption
0,ac_01,2025-04-16 11:58:08,0.00
1,ac_02,2025-04-16 11:58:08,0.00
2,ac_03,2025-04-16 11:58:08,346.88
3,ac_04,2025-04-16 11:58:08,247.79
4,mw_01,2025-04-16 11:58:08,0.00
...,...,...,...
26203,B_06,2025-05-07 11:28:08,0.00
26204,B_07,2025-05-07 11:28:08,0.00
26205,B_08,2025-05-07 11:28:08,0.00
26206,B_09,2025-05-07 11:28:08,0.00


In [52]:
data['predictions'].head(60)

,timestamp,power_consumption
